In [1]:
import urllib.request
import zipfile
from lxml import etree
import re

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

import gensim
from gensim.models import KeyedVectors

# nltk : 3.2.5  |  gensim : 3.6.0
print(f'\n>>> nltk : {nltk.__version__}  |  gensim : {gensim.__version__}') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

>>> nltk : 3.2.5  |  gensim : 3.6.0
